### Install pyfor

1. Create a conda environment for this project. You should have installed conda or miniconda already for this to run.
```
conda env create -f ./3dworkbench.yml
```
2. Install pyfor library and dependancies. This will take a while due to dependancies heavy files.

3. If necessary, convert the LAZ file to LAS (You can do this using the LASTools Plugin on QGIS)

4. Follow [this](https://geonetcast.wordpress.com/2022/05/12/creating-a-geotiff-from-a-numpy-array/) for exporting np array to GeoTiff.

## Import Libraries

In [ ]:
import laspy
import open3d as o3d
import numpy as np
import trimesh as tm
import shapely
import geopandas
import rasterio
from osgeo import gdal, osr, ogr # Python bindings for GDAL
import matplotlib.pyplot as plt
import pickle

### Import Files

In [ ]:
las = laspy.read('../Data/OldCityEnschede.las')

### Transform data into a Open3D point cloud and visualize

In [ ]:
point_data = np.stack([las.X, las.Y, las.Z], axis=0).transpose((1,0))

In [ ]:
geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(point_data)

In [ ]:
o3d.visualization.draw_geometries([geom])

#### convert open3D object into a np array and transform into meters and create a new point cloud


In [ ]:
## convert open3D object into a np array and transform into meters and create a new point cloud
xyz_load = (np.asarray(geom.points))/1000
newGeom= o3d.geometry.PointCloud()
newGeom.points = o3d.utility.Vector3dVector(xyz_load)

In [ ]:
#get boundaries and size of the diagonal

minb= np.min(xyz_load, axis=0)
maxb= np.max(xyz_load, axis=0)

diagonal = (maxb - minb)
diagonal

In [ ]:
# create bounding box
bbox = (minb,maxb)
bbox

#### define Pixel size and bounding Box

In [ ]:
pixsize =np.array([5,5])
bboxR2 = ([minb[0],minb[1]],[maxb[0],maxb[1]])
bboxR2

In [ ]:
bboxN2 = bboxR2/pixsize
bboxN2

In [ ]:
bboxN2=np.rint(bboxN2).astype(int)
bboxN2

In [ ]:
diagonalN2=bboxN2[1]-bboxN2[0]
diagonalN2 

(m,n)=diagonalN2
m,n

In [ ]:
# Initialize empty lists and arrays to store pixel-related information
pixelsR2 = []       # List to store 2D pixel coordinates in the original image
pixelPolygons = []  # List to store shapely.geometry.box polygons for each pixel
pixelsR3 = np.zeros((m, n, 3))  # 3D array to store the transformed pixel coordinates and average height
heightMap = np.zeros((m, n))    # 2D array to store the average height of each pixel
DEMVertices = []    # List to store 3D coordinates of each pixel

# Loop through each pixel in the image
for i in range(m):
    for j in range(n):
        
        # Calculate 2D pixel coordinates in the original image
        pixelN2 = [i, j]
        pixelR2 = (pixelN2 * pixsize) + bboxR2[0]
        
        # Store the 2D pixel coordinates
        pixelsR2.append(pixelR2)
        
        # Calculate the bounding box of the pixel in 3D space
        halfDiagonal = 0.5 * pixsize
        pixelBBBl = (pixelR2 - halfDiagonal).reshape(-1)
        pixelBBTr = (pixelR2 + halfDiagonal).reshape(-1)
        minBound = np.append(pixelBBBl, minb[2])
        maxBound = np.append(pixelBBTr, maxb[2])
        bounds = [minBound, maxBound]
        
        # Create an Open3D point cloud for the bounding box
        tempPoints = o3d.geometry.PointCloud()
        tempBounds = tempPoints.points = o3d.utility.Vector3dVector(bounds)
        
        # Create an axis-aligned bounding box for the pixel
        smallBB = o3d.geometry.AxisAlignedBoundingBox.create_from_points(tempBounds)
        
        # Create a shapely polygon for the pixel
        pixelBB = (pixelBBBl[0], pixelBBBl[1], pixelBBTr[0], pixelBBTr[1])
        pixelPolygon = shapely.geometry.box(*pixelBB, ccw=True)
        pixelPolygons.append(pixelPolygon)
        
        # Crop the geometry using the bounding box
        pixelPC = newGeom.crop(smallBB)
        pcArray = np.asarray(pixelPC.points)
        
        # Calculate the average height of the points within the pixel
        averageHeight = 0
        if pcArray.shape[0] > 0:
            averageHeight = np.average(pcArray, axis=0)[2]
        
        # Transform pixel coordinates and store in 3D array
        j = n - 1 - j
        pixelR3 = np.array([*pixelR2, averageHeight])
        pixelsR3[i, j] = pixelR3
        
        # Store the average height in the height map
        heightMap[i, j] = averageHeight
        
        # Store the 3D coordinates in the list
        DEMVertices.append(pixelR3)

### Transpose the matrix and display using Matplotlib

In [ ]:
heightMap=heightMap.transpose()
plt.matshow(heightMap)
heightMap

In [ ]:
# Convert the list of pixel coordinates (pixelsR2) to a NumPy array
points = np.array(pixelsR2)

# Display the shape of the array (number of rows and columns)
points.shape

# Reshape the array to have the number of rows equal to the original shape's first dimension
points = points.reshape(points.shape[0], points.shape[1])

# Display the reshaped array
points

In [ ]:
# Append a column of zeros to the 2D pixel coordinates array, creating a 3D array
points3D = np.c_[points, np.zeros(points.shape[0])]

# Display the resulting 3D array
points3D

In [ ]:
#vizulise points of the new array.

geom = o3d.geometry.PointCloud()
geom.points = o3d.utility.Vector3dVector(points3D)
o3d.visualization.draw_geometries([geom])

In [ ]:
# Define a function to calculate the geotransform for raster data
def getGeoTransform(extent, nlines, ncols):
    # Calculate the pixel size in the x and y directions
    resx = (extent[2] - extent[0]) / ncols
    resy = (extent[3] - extent[1]) / nlines
    
    # Return the geotransform parameters as a list
    return [extent[0], resx, 0, extent[3], 0, -resy]

# Define the data extent (min. lon, min. lat, max. lon, max. lat) using bboxR2
extent = [*bboxR2[0], *bboxR2[1]]
extent


In [ ]:
# Export the test array to GeoTIFF ================================================
 
# Get GDAL driver GeoTiff
driver = gdal.GetDriverByName('GTiff')
 
# Get dimensions
nlines = heightMap.shape[0]
ncols = heightMap.shape[1]
nbands = 1
data_type = gdal.GDT_Float32

In [ ]:
# Create a temp grid
#options = ['COMPRESS=JPEG', 'JPEG_QUALITY=80', 'TILED=YES']
grid_data = driver.Create('height_map', ncols, nlines, 1, data_type)#, options)
 
# Write data for each bands
grid_data.GetRasterBand(1).WriteArray(heightMap)

# Lat/Lon WSG84 Spatial Reference System
srs = osr.SpatialReference()
srs.SetFromUserInput("EPSG:28992") #RD_New EPSG code

In [ ]:
# Setup projection and geo-transform
grid_data.SetProjection(srs.ExportToWkt())
grid_data.SetGeoTransform(getGeoTransform(extent, nlines, ncols))
 
# Save the file
file_name = '../Results/height_map.tif'
print(f'Generated GeoTIFF: {file_name}')
driver.CreateCopy(file_name, grid_data, 0)  

# Close the file
driver = None
grid_data = None
 
# Delete the temp grid
import os                
os.remove('height_map')


In [ ]:
# Initialize lists to store vertex indices and face lists
vertexIndices = []
faceList = []

# Loop through each vertex in the mesh grid
for i in range(m - 1):
    for j in range(n - 1):
        # Calculate vertex indices for the current quad
        v0 = i * n + j
        v1 = v0 + 1
        v2 = v0 + n
        v3 = v1 + n
        
        # Define triangles for the current quad
        tbl = [v0, v2, v1]
        ttr = [v1, v2, v3]
        
        # Append triangles to the face list
        faceList.append(tbl)
        faceList.append(ttr)

# Convert the face list to a NumPy array
faceArray = np.array(faceList)
faceArray

In [ ]:
# Now visualize the mesh using Trimesh

mesh=tm.Trimesh(vertices=DEMVertices,faces=faceArray)
mesh.show()

### Save the mesh as a pickle file to use in the next steps

In [ ]:
with open('mesh.pkl', 'wb') as file:
    pickle.dump(mesh, file)

In [ ]:
# tempMesh = mesh.copy()
f=mesh.export(file_obj='../Results/mesh.obj',file_type='obj')
f